In [1]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=0)

## Day 1

In part 2, increment of sum of three consecutive numbers is determined from the sliding window of **four** elements. The head of the four elements is going out, and the tail element is incomming.  

In [14]:
# day 1, part 1

import itertools
# python 3.10 introduces itertools.pairwise

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for a, b in itertools.pairwise(xs) if a < b)
    print(count)

1752


In [15]:
# day 1, part 2

import itertools
import collections
from typing import Iterable, Generator, Any

# https://docs.python.org/3/library/itertools.html#itertools-recipes
def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) -> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(itertools.islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for head, _, _, tail in sliding_window(xs, 4) if head < tail)
    print(count)

1781


## Day 2

Discuss change in submarine's position both horizontally and vertically (as depth). The part 2 introduces "aim" as an additional variable.

In [3]:
from typing import Iterable

def day02_part1(xs: Iterable[tuple[str, int]]) -> int:
    """
    """
    h, d = 0, 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
        elif cmd == "down":
            d += amount
        elif cmd == "up":
            d -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [5]:
test = """
forward 5
down 5
forward 8
up 3
down 8
forward 2
"""
xs = [s.split() for s in test.split("\n") if s.strip()]
xs = [(cmd, int(w)) for cmd, w in xs]
day02_part1(xs)

150

In [6]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part1(xs)
    print(res)

1804520


In [7]:
def day02_part2(xs: Iterable[tuple[str, int]]) -> int:
    h = 0
    d = 0
    aim = 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
            d += amount * aim
        elif cmd == "down":
            aim += amount
        elif cmd == "up":
            aim -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [8]:
test = """
forward 5
down 5
forward 8
up 3
down 8
forward 2
"""
xs = [s.split() for s in test.split("\n") if s.strip()]
xs = [(cmd, int(w)) for cmd, w in xs]
day02_part2(xs)

900

In [9]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part2(xs)
    print(res)


1971095320


## Day 3: Binary manipulations

In [41]:
import collections
from typing import Iterable


def day03_part1(content: str) -> int:

    def most_common(xs: Iterable[str]) -> str:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return str(int(cnt['0'] <= cnt['1']))
    
    lines = [list(line.strip()) for line in content.split() if line.strip()]
    gamma = "".join(most_common(seq) for seq in zip(*lines))
    gamma = int(gamma, 2)
    bin_length = len(lines[0])
    epsilon = ((1 << bin_length) - 1) ^ gamma
    print(f"{(gamma, epsilon) = }")
    return gamma * epsilon


In [42]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part1(s)

(gamma, epsilon) = (22, 9)


198

In [43]:
with open("./2021/day03_input.txt") as f:
    res = day03_part1(f.read())
    print(res)

(gamma, epsilon) = (190, 3905)
741950


In [44]:
import numpy as np
from typing import Iterable
import collections

def day03_part2(content: str) -> int:

    def most_common(xs: Iterable[int]) -> int:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return int(cnt[0] <= cnt[1])
    

    def _rating(zs, n_digits, cmp) -> int:
        xs = np.copy(zs)
        
        i = n_digits - 1
        while len(xs) > 1:
            digits = (xs >> i) & 1
            criteria = cmp(digits, most_common(digits))
            xs = xs[criteria]
            i -= 1
        return xs[0]

    def oxygen_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.equal)

    def co2_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.not_equal)
    
    lines = [line.strip() for line in content.split() if line.strip()]
    xs = np.array([int(line, 2) for line in lines])

    n_digits = len(lines[0])
    oxygen = oxygen_rating(xs, n_digits)
    co2 = co2_rating(xs, n_digits)
    print(f"{(oxygen, co2) = }")
    return oxygen * co2


In [45]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part2(s)

(oxygen, co2) = (23, 10)


230

In [46]:
with open("./2021/day03_input.txt") as f:
    res = day03_part2(f.read())
    print(res)

(oxygen, co2) = (282, 3205)
903810


## Day 4: 5 x 5 Bingo

**NOTE**: diagonals don't count!

In [133]:
from typing import Generator, Any, Iterable, Optional
import collections


def day04_parse(content: str) -> tuple[list[int], list[list[list[int]]]]:
    """Return integer sequence AND a list of 5x5 bingos"""
    chunks = [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]
    xs = [int(s) for s in chunks[0].split(",") if s.strip()]
    dominos = [
        [
            [int(s) for s in line.split()]
            for line in chunk.split("\n") if line.strip()
        ] 
        for chunk in chunks[1:]
    ]
    return xs, dominos


def day04_candidates(domino: list[list[Any]]) -> Generator[frozenset[Any], None, None]:
    """
    """
    for xs in domino:
        yield frozenset(xs)
    
    for xs in zip(*domino):
        yield frozenset(xs)

def day04_part1(calls: Iterable[int], dominos: list[list[list[int]]]) -> Optional[int]:

    d = collections.defaultdict(list)
    for i, domino in enumerate(dominos):
        for candidate in day04_candidates(domino):
            d[candidate].append(i)
        
    called = set()
    last_x = -1   # placeholder for scope purpose
    for x in calls:
        called.add(x)
        if any(candidate <= called for candidate in d):
            last_x = x
            break

    for candidate in d:
        if candidate <= called:
            idx = d[candidate][0]  # assume the winner is unique
            board = frozenset({x for xs in dominos[idx] for x in xs})
            print(f"{idx = },  {last_x = },  {candidate = },  {called = }")
            return sum(board - called) * last_x
        
    print("something is wrong!")
    return None

In [134]:
s = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

calls, dominos = day04_parse(s)
day04_part1(calls, dominos)

idx = 2,  last_x = 24,  candidate = frozenset({4, 14, 17, 21, 24}),  called = {0, 2, 4, 5, 7, 9, 11, 14, 17, 21, 23, 24}


4512

In [135]:
with open("./2021/day04_input.txt") as f:
    calls, dominos = day04_parse(f.read())
    res = day04_part1(calls, dominos)
    print(res)

idx = 88,  last_x = 14,  candidate = frozenset({71, 8, 14, 23, 24}),  called = {3, 8, 9, 12, 14, 18, 22, 23, 24, 30, 37, 40, 45, 53, 54, 57, 58, 62, 70, 71, 72, 73, 80, 81, 88, 95, 98}
10374


In [139]:
def day04_part2(calls: Iterable[int], dominos: list[list[list[int]]]) -> Optional[int]:

    d = collections.defaultdict(list)
    for i, domino in enumerate(dominos):
        for candidate in day04_candidates(domino):
            d[candidate].append(i)
    
    n_boards = len(dominos)
    print(f"{n_boards = }")
    won_indices = []
    called: set[int] = set()
    done = set()
    last_x = -1

    for x in calls:
        called.add(x)
        for candidate, indices in d.items():
            if candidate not in done and candidate <= called:
                print(set(candidate), indices)
                last_x = x
                for idx in indices:
                    if idx not in won_indices:
                        won_indices.append(idx)
                done.add(candidate)
    
        if len(won_indices) == n_boards:
            break

    if won_indices:
        idx = won_indices[-1]
        board = frozenset({x for xs in dominos[idx] for x in xs})
        print(f"{idx = },  {last_x = }, {called = }")
        return sum(board - called) * last_x
    
    print("something is wrong!")
    return None

In [140]:
s = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

calls, dominos = day04_parse(s)
day04_part2(calls, dominos)

n_boards = 3
{17, 4, 21, 24, 14} [2]
{16, 21, 7, 9, 14} [0]
{0, 16, 7, 10, 13} [1]
idx = 1,  last_x = 13, called = {0, 2, 4, 5, 7, 9, 10, 11, 13, 14, 16, 17, 21, 23, 24}


1924

In [141]:
with open("./2021/day04_input.txt") as f:
    calls, dominos = day04_parse(f.read())
    res = day04_part2(calls, dominos)
    print(res)

n_boards = 100
{23, 71, 8, 24, 14} [88]
{0, 34, 55, 45, 95} [31]
{34, 30, 8, 29, 14} [46]
{0, 66, 88, 12, 29} [62]
{3, 22, 9, 58, 27} [52]
{55, 88, 73, 27, 28} [48]
{53, 23, 72, 28, 45} [81]
{66, 19, 37, 87, 24} [36]
{34, 82, 54, 87, 14} [77]
{29, 83, 71, 45, 14} [9]
{83, 55, 73, 58, 9} [10]
{80, 48, 66, 83, 58} [48]
{96, 36, 23, 88, 31} [20]
{98, 36, 53, 88, 29} [35]
{18, 82, 54, 22, 26} [9]
{81, 8, 24, 26, 31} [97]
{36, 9, 63, 14, 95} [4]
{36, 71, 8, 9, 63} [21]
{66, 19, 23, 31, 63} [60]
{21, 22, 14, 73, 9} [9]
{37, 21, 71, 22, 73} [17]
{96, 19, 21, 70, 63} [86]
{82, 53, 21, 72, 9} [86]
{33, 3, 87, 73, 28} [3]
{80, 33, 81, 70, 62} [23]
{48, 33, 36, 70, 40} [61]
{33, 70, 12, 46, 95} [84]
{83, 70, 86, 87, 24} [1]
{26, 36, 86, 58, 29} [9]
{81, 66, 86, 55, 62} [48]
{32, 19, 37, 21, 45} [0]
{32, 98, 86, 9, 63} [9]
{32, 82, 71, 24, 12} [9]
{32, 66, 21, 9, 62} [17]
{32, 96, 81, 5, 53} [40]
{0, 34, 5, 56, 63} [23]
{98, 37, 56, 95, 63} [50]
{33, 98, 86, 56, 26} [56]
{48, 71, 72, 56, 29} [81]


## Day 5: いもす法

In [183]:
import collections
Coord = tuple[int, int]

def day05_parse(content: str) -> list[tuple[Coord, Coord]]:
    lines = [line.strip().split(" -> ") for line in content.split("\n") if line.strip()]
    res = []
    for from_, to_ in lines:
        x1, y1 = map(int, from_.split(","))
        x2, y2 = map(int, to_.split(","))
        res.append(((x1, y1), (x2, y2)))
    return res


def day05_part1(pairs: list[tuple[Coord, Coord]]) -> int:
    x_ub = max(max(x1, x2) for (x1, _), (x2, _) in pairs)
    y_ub = max(max(y1, y2) for (_, y1), (_, y2) in pairs)

    grid_v = collections.defaultdict(int)
    grid_h = collections.defaultdict(int)
    for (x1, y1), (x2, y2) in pairs:
        if y1 == y2:
            grid_v[min(x1, x2), y1] += 1
            grid_v[max(x1, x2) + 1, y2] -= 1
        elif x1 == x2:
            grid_h[x1, min(y1, y2)] += 1
            grid_h[x2, max(y1, y2) + 1] -= 1    

    grid = collections.defaultdict(int)
    cnt = 0
    for i in range(x_ub + 2):
        for j in range(y_ub + 2):
            grid_v[i, j] += grid_v[i - 1, j]
            grid_h[i, j] += grid_h[i, j - 1]
            grid[i, j] = grid_v[i, j] + grid_h[i, j]
            if grid[i, j] > 1:
                cnt += 1
    return cnt


def day05_display(grid):
    x_ub = max(x for (x, _) in grid.keys())
    y_ub = max(y for (_, y) in grid.keys())

    s = "\n".join(
        " ".join(str(grid[i, j]) for i in range(x_ub + 2))
        for j in range(y_ub + 2)
    )
    
    print(s)
    print()

In [184]:
content = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""

pairs = day05_parse(content)
day05_part1(pairs)

5

In [185]:
import math

def day05_part2(pairs: list[tuple[Coord, Coord]], show: bool=False) -> int:
    x_ub = max(max(x1, x2) for (x1, _), (x2, _) in pairs)
    y_ub = max(max(y1, y2) for (_, y1), (_, y2) in pairs)

    grid = collections.defaultdict(int)
    for (x1, y1), (x2, y2) in pairs:
        d = math.gcd(x2 - x1, y2 - y1)
        for i in range(d + 1):
            x = x1 + i * (x2 - x1) // d
            y = y1 + i * (y2 - y1) // d
            grid[x, y] += 1

    cnt = 0
    for i in range(x_ub + 2):
        for j in range(y_ub + 2):
            if grid[i, j] > 1:
                cnt += 1
    
    if show:
        day05_display(grid)
    return cnt


In [186]:
content = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""

pairs = day05_parse(content)
day05_part2(pairs, show=True)

1 0 1 0 0 0 0 1 1 0 0 0
0 1 1 1 0 0 0 2 0 0 0 0
0 0 2 0 1 0 1 1 1 0 0 0
0 0 0 1 0 2 0 2 0 0 0 0
0 1 1 2 3 1 3 2 1 1 0 0
0 0 0 1 0 2 0 0 0 0 0 0
0 0 1 0 0 0 1 0 0 0 0 0
0 1 0 0 0 0 0 1 0 0 0 0
1 0 0 0 0 0 0 0 1 0 0 0
2 2 2 1 1 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0



12

In [182]:
with open("./2021/day05_input.txt") as f:
    pairs = day05_parse(f.read())
    res = day05_part2(pairs)
    print(res)

21101


## Day 6: Lanternfish
Let n[t, r] be the number of lanternfish with remainder count r at time t. Consider change in state from (t-1), we have

```
n[t, 8] = n[t - 1, 0]
n[t, 7] = n[t - 1, 8]
n[t, 6] = n[t - 1, 7] + n[t - 1, 0]
n[t, r] = n[t - 1, r + 1]    (for 0 <= r <= 5)
```

In [35]:
import collections

def day06_f(n: dict[tuple[int, int], int], t: int):
    n[t, 8] = n[t - 1, 0]
    n[t, 7] = n[t - 1, 8]
    n[t, 6] = n[t - 1, 7] + n[t -1, 0]
    for i in range(6):
        n[t, i] = n[t - 1, i + 1]

def day06(xs: list[int], days=80) -> int:
    d = collections.defaultdict(int)

    counts = collections.Counter(xs)
    for r, count in counts.items():
        d[0, r] = count
    
    for t in range(1, days + 1):
        day06_f(d, t)
    
    # for t in range(19):
    #     print(f"{t}: {[d[t, r] for r in range(9)]}")

    res = sum(d[days, r] for r in range(9))
    return res

In [36]:
content = "3,4,3,1,2"
xs = [int(c) for c in content.split(",")]
day06(xs, 80)

5934

In [37]:
with open("./2021/day06_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day06(xs, days=80)
    print(res)

362740


In [38]:
with open("./2021/day06_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day06(xs, days=256)
    print(res)

1644874076764


## Day 7: Crab positions

In part 1, minimum location p satisfying $\sum_i |p - x_i|$ is given from the median of $x_i$ (with $i = 0, 1, 2, \dots$). 

In [44]:
from typing import Union

def day07_median(xs: list[int]) -> int:
    """Find median of xs. If the number of xs is even, pick smaller one."""
    n = len(xs)
    k = n // 2
    xs.sort()
    return xs[k]

def day07_part1(xs: list[int]) -> Union[int, float]:
    """Find median of xs"""
    med = day07_median(xs)
    res = sum(abs(med - x) for x in xs)
    return res

In [45]:
xs = [16,1,2,0,4,2,7,1,2,14]
day07_part1(xs)

37

In [76]:
with open("./2021/day07_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day07_part1(xs)
    print(res)

344735


In Part 2, the cost $f(n)$ of $n$ steps is $f(n) = 1 + 2 + ... + n = \frac{n (n + 1)}{2}$.

Find position $p$ minimizing $\sum_i f(\lvert p - x_i \rvert)$, which is to find $p$ minimizing  $\sum_i \left[ (p - x_i)^2 + \lvert p - x_i \rvert \right] $. Hence

$$
\frac{d f}{d p} = 2 \sum_i (p - x_i) + \sum_i \mathrm{sgn}(p - x_i) = 0
$$

Assuming the funciton $\frac{d f}{d p} = 2 N p - \sum_i x_i + \sum_i \mathrm{sgn} (p - x_i)$ is monotonically increasing, find $p$ satisfying $g(p) = 0$ with binary search. 

Note that the solution of the binary search $p_\mathrm{sol}$ is an integer satisfying $g(p_\mathrm{sol}) \geq 0$. It is possible that $p = (p_\mathrm{sol} - 1)$ giving the minimum $f(p)$ if the inequality holds.

In [125]:
import bisect

def binary_search(xs):
    def is_ok(index):
        return day07_dfdp(index, xs) >= 0

    ng = -1
    ok = 1000000

    while abs(ok - ng) > 1:
        mid = (ok + ng) // 2
        if is_ok(mid):
            ok = mid
        else:
            ng = mid
    return ok

def day07_sgn_sum(p, xs):
    """Return sum_i sgn(p - x_i). Assume xs is sorted."""
    n = len(xs)
    idx_left = bisect.bisect_left(xs, p)
    idx_right = bisect.bisect_right(xs, p)
    # print(f"{idx_left = }")
    # print(f"{idx_right = }")
    return  idx_left + idx_right - n

def day07_dfdp(p: int, xs: list[int]):
    np = len(xs) * p
    sum_xs = sum(xs)
    sum_sgn = day07_sgn_sum(p, xs)
    return 2 * (np - sum_xs) + sum_sgn


def day07_cost(p: int, xs: list[int]) -> int:
    res = 0
    for x in xs:
        d = abs(p - x)
        res += d * (d + 1) // 2
    return res    

def day07_part2(xs: list[int]):
    xs.sort()
    p = binary_search(xs)
    res = day07_cost(p, xs)
    if (tmp := day07_cost(p - 1, xs)) < res:
        res = tmp
        p = p - 1
    print(f"{p = }")

    return res

In [126]:
xs = [16,1,2,0,4,2,7,1,2,14]
day07_part2(xs)

p = 5


168

In [127]:
with open("./2021/day07_input.txt") as f:
    xs = [int(c) for c in f.read().split(",")]
    res = day07_part2(xs)
    print(res)

p = 474
96798233


## Day 8: Seven-segment digit display

```
0: abcefg  (6 segments ON)
1: cf      (2 segments ON)      
2: acdeg   (5 segments ON)      
3: acdfg   (5 segments ON)      
4: bcdf    (4 segments ON)      
5: abdfg   (5 segments ON)      
6: abdefg  (6 segments ON)      
7: acf     (3 segments ON)      
8: abcdefg (7 segments ON)      
9: abcdfg  (6 segments ON)      
```

* Find 1, 4, 7, 8 from their unique length, respectively.
* 6 is identified as having length 6 AND not containing digit 1 representation.
* 9 is identified as the only item containing 4 (bcdf) AND length 6.
* 0 is identified from the rest of string of length 6.

* 'e' is found from difference of 9 and 8.
* 2 is identified as the only the item with length 5 and containing 'e'.
* 5 is identified as the set difference 6 - {'e'}.
* 3 is identified as the rest of the entry with length 5.


In [64]:
import itertools
import collections

def day08_parse(content: str) -> list[tuple[list[str], list[str]]]:
    pairs = [line.strip().split(" | ") for line in content.split("\n") if line.strip()]
    res = [(first.split(), second.split()) for first, second in pairs]
    return res

def day08_count(entries: list[tuple[list[str], list[str]]]) -> int:
    def is_1_4_7_8(s: str) -> bool:
        return len(s) in [2, 3, 4, 7]

    iterable = itertools.chain.from_iterable(b for (_, b) in entries)
    return sum(1 for s in iterable if is_1_4_7_8(s))


def day08_decode(training: list[str], inference: list[str]) -> int:
    xs = [frozenset(x) for x in training]
    assert collections.Counter(map(len, xs)) == collections.Counter({2: 1, 3: 1, 4: 1, 7: 1, 5: 3, 6: 3})
    
    d = dict()  # representation of digits
    d[1] = next(x for x in xs if len(x) == 2)
    d[4] = next(x for x in xs if len(x) == 4)
    d[7] = next(x for x in xs if len(x) == 3)
    d[8] = next(x for x in xs if len(x) == 7)

    d[6] = next(x for x in xs if len(x) == 6 and (not d[1] < x))
    d[9] = next(x for x in xs if len(x) == 6 and d[4] < x)
    d[0] = next(x for x in xs if len(x) == 6 and x not in (d[6], d[9]))

    diff = d[8] - d[9]
    assert len(diff) == 1
    letter_e = next(iter(diff))

    d[2] = next(x for x in xs if len(x) == 5 and letter_e in x)
    d[5] = next(x for x in xs if x == (d[6] - {letter_e}))
    d[3] = next(x for x in xs if len(x) == 5 and x not in (d[2], d[5]))

    inv_dict = {d[i]: str(i) for i in range(10)}
    digits = "".join(inv_dict[frozenset(x)] for x in inference)
    return int(digits)


def day08(content: str) -> int:
    entries = day08_parse(content)
    res = sum(day08_decode(training, inference) for training, inference in entries)
    return res



In [65]:
s = """
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
"""
entries = day08_parse(s)
res = day08_count(entries)
print(res)

26


In [66]:
with open("./2021/day08_input.txt") as f:
    entries = day08_parse(f.read())
    res = day08_count(entries)
res

495

In [67]:
training = "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab".split()
inference = "cdfeb fcadb cdfeb cdbaf".split()
day08_decode(training, inference)

5353

In [68]:
with open("./2021/day08_input.txt") as f:
    res = day08(f.read())
res

1055164

## Day 9: Heatmap

In part 1, you find local minima scanning 3x3 blocks. One may look at 2x3 blocks at the edges, and 2x2 block at the corners. Or we can do convolution calculation with 1-padding with max cell values, say 9.

In part 2, you find basins defined by contiguous cells NOT equal to 9. Find the three largest basins and get their sum. It can be done with breath-first or depth-first search.

In [151]:
import numpy as np
import numpy.typing as npt

def day09_parse(content: str) -> npt.NDArray[np.int_]:
    raw = np.array([
        np.array(list(line.strip()), dtype=int) for line in content.split("\n") if line.strip()
    ])
    res = np.pad(raw, 1, 'constant', constant_values=9)
    return res

def day09_part1(grid: npt.NDArray[np.int_]) -> tuple[int, list[tuple[int, int]]]:

    def is_lowest(block: npt.NDArray[np.int_]) -> bool:
        assert (3, 3) == block.shape
        surrounding = min(block[0].min(), block[2].min(), block[1, 0], block[1, 2])
        center = block[1, 1]
        return center < surrounding

    m, n = grid.shape
    m -= 2  # exclude pads
    n -= 2  # exclude pads

    risk_level = 0
    local_minima = []
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            block = grid[i - 1: i + 2, j - 1: j + 2]
            if is_lowest(block):
                risk_level += grid[i, j] + 1
                local_minima.append((i, j))
    return risk_level, local_minima


def day09_basin(grid: npt.NDArray[np.int_], start: tuple[int, int]) -> int:
    seen = set()
    stack = [start]
    i_ub, j_ub = grid.shape

    # print(f"start with {stack = }")

    while stack:
        cell = stack.pop()
        if cell in seen:
            continue
        seen.add(cell)

        (i, j) = cell
        candidates = [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        for (i_next, j_next) in candidates:
             if (0 <= i_next < i_ub) and (0 <= j_next < j_ub) and grid[i_next, j_next] < 9:
                 stack.append((i_next, j_next))

    # print(f"{seen = },    {len(seen) = }")
    return len(seen)


def day09_part2(grid: npt.NDArray[np.int_]) -> int:
    _, local_minima = day09_part1(grid)
    basin_sizes = [day09_basin(grid, start) for start in local_minima]
    basin_sizes.sort(reverse=True)
    print(f"{basin_sizes = }")
    return np.prod(basin_sizes[:3])


In [152]:
s = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""
grid = day09_parse(s)
res, _ = day09_part1(grid)
res

15

In [153]:
with open("./2021/day09_input.txt") as f:
    grid = day09_parse(f.read())
    res, _ = day09_part1(grid)

res

522

In [154]:
s = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""
grid = day09_parse(s)
res = day09_part2(grid)
res

basin_sizes = [14, 9, 9, 3]


1134

In [157]:
with open("./2021/day09_input.txt") as f:
    grid = day09_parse(f.read())
    res = day09_part2(grid)

res

basin_sizes = [106, 94, 92, 91, 90, 90, 89, 88, 87, 87, 85, 82, 82, 81, 79, 77, 76, 76, 75, 75, 72, 71, 70, 70, 69, 69, 69, 68, 67, 67, 67, 66, 64, 63, 62, 62, 62, 62, 60, 60, 59, 58, 58, 58, 57, 57, 56, 56, 56, 55, 55, 53, 51, 50, 50, 50, 50, 49, 49, 48, 47, 47, 47, 47, 47, 46, 46, 46, 45, 45, 44, 44, 44, 44, 43, 42, 42, 41, 41, 39, 39, 39, 38, 38, 37, 37, 36, 35, 35, 35, 34, 34, 33, 33, 33, 32, 31, 30, 30, 30, 30, 29, 29, 29, 28, 28, 27, 27, 27, 26, 26, 25, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 21, 21, 20, 19, 19, 19, 19, 18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 16, 16, 15, 15, 15, 15, 15, 14, 14, 14, 14, 13, 13, 12, 12, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


916688

## Day 10: Brackets

Use stack to check matching brackets.

In [11]:
def day10_corrupted(s: str) -> int:
    opening = "([{<"
    closing = ")]}>"
    matching = dict(zip(opening, closing))
    points = {')': 3, ']': 57, '}': 1197, '>': 25137}
    stack = []


    for c in s:
        if c in opening:
            stack.append(c)
        else:
            assert stack
            latest = stack.pop()
            if c != matching[latest]:
                return points[c]
    return 0
    

In [12]:
s = """
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
"""

xs = [line.strip() for line in s.split("\n") if line.strip()]
sum(day10_corrupted(x) for x in xs)

26397

In [13]:
with open("./2021/day10_input.txt") as f:
    xs = [line.strip() for line in f.readlines() if line.strip()]
    res = sum(day10_corrupted(x) for x in xs)
res

394647

In [14]:
import functools

def day10_incomplete(s: str) -> int:
    opening = "([{<"
    closing = ")]}>"
    matching = dict(zip(opening, closing))
    stack = []

    for c in s:
        if c in opening:
            stack.append(c)
        else:
            assert stack
            latest = stack.pop()
            assert c == matching[latest]
    

    point = {')': 1, ']': 2, '}': 3, '>': 4}
    missing = [matching[x] for x in reversed(stack)]
    res = functools.reduce(lambda acc, x: acc * 5 + point[x], missing, 0)
    return res


def day10_part2(s: str) -> int:
    xs = [line.strip() for line in s.split("\n") if line.strip()]
    xs = [x for x in xs if day10_corrupted(x) == 0]
    points = [day10_incomplete(x) for x in xs]
    k = len(points) // 2
    points.sort()
    return points[k]


In [15]:
s = """
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
"""

day10_part2(s)

288957

In [16]:
with open("./2021/day10_input.txt") as f:
    res = day10_part2(f.read())
res

2380061249

## Day 11: Flashes in 2-D grid

[TIL] `indexing="ij"` option in `np.meshgrid` makes its behavior sane though it becomes incompatible with Matlab.

In [93]:
import numpy as np
import numpy.typing as npt

Grid = npt.NDArray[np.int_]
# PADVAL = np.iinfo(np.int64).min
PADVAL = -1000000

def day11_parse(s: str) -> Grid:
    grid = np.array([list(line.strip()) for line in s.split("\n") if line.strip()], dtype=int)
    grid = np.pad(grid, 1, 'constant', constant_values=PADVAL)
    return grid


def day11_update(grid: Grid):
    m, n = grid.shape
    i_idx, j_idx = np.meshgrid(range(m), range(n), indexing="ij")
    seen = np.full_like(grid, False, dtype=bool)
    grid += 1

    while (cond := (grid > 9) & np.logical_not(seen)).any():
        seen |= cond
        ij_pairs = np.stack([i_idx[cond], j_idx[cond]], axis=1)
        for i, j in ij_pairs:
            for delta_i in (-1, 0, 1):
                for delta_j in (-1, 0, 1):
                    grid[i + delta_i, j + delta_j] += 1
    
    grid[grid > 9] = 0
    sparked = np.sum(grid == 0)

    return sparked


def day11_display(grid: Grid):
    mat = grid[1:-1, 1:-1]
    s = "\n".join(" ".join(map(str, xs)) for xs in mat)
    print()
    print(s)


def day11_part1(s: str) -> int:
    grid = day11_parse(s)
    sparks = sum(day11_update(grid) for _ in range(100))
    return sparks


def day11_part2(s: str) -> int:
    grid = day11_parse(s)
    m, n = grid.shape
    m -= 2  # remove padding
    n -= 2  # remove padding
    t = 0
    while True:
        t += 1
        spark = day11_update(grid)
        if spark == m * n:
            break
    return t

In [96]:
s = """
11111
19991
19191
19991
11111
"""

grid = day11_parse(s)
day11_display(grid)

day11_update(grid)
day11_display(grid)

day11_update(grid)
day11_display(grid)



1 1 1 1 1
1 9 9 9 1
1 9 1 9 1
1 9 9 9 1
1 1 1 1 1

3 4 5 4 3
4 0 0 0 4
5 0 0 0 5
4 0 0 0 4
3 4 5 4 3

4 5 6 5 4
5 1 1 1 5
6 1 1 1 6
5 1 1 1 5
4 5 6 5 4


In [97]:
s = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""

grid = day11_parse(s)
day11_display(grid)

day11_update(grid)
day11_display(grid)

day11_update(grid)
day11_display(grid)


5 4 8 3 1 4 3 2 2 3
2 7 4 5 8 5 4 7 1 1
5 2 6 4 5 5 6 1 7 3
6 1 4 1 3 3 6 1 4 6
6 3 5 7 3 8 5 4 7 8
4 1 6 7 5 2 4 6 4 5
2 1 7 6 8 4 1 7 2 1
6 8 8 2 8 8 1 1 3 4
4 8 4 6 8 4 8 5 5 4
5 2 8 3 7 5 1 5 2 6

6 5 9 4 2 5 4 3 3 4
3 8 5 6 9 6 5 8 2 2
6 3 7 5 6 6 7 2 8 4
7 2 5 2 4 4 7 2 5 7
7 4 6 8 4 9 6 5 8 9
5 2 7 8 6 3 5 7 5 6
3 2 8 7 9 5 2 8 3 2
7 9 9 3 9 9 2 2 4 5
5 9 5 7 9 5 9 6 6 5
6 3 9 4 8 6 2 6 3 7

8 8 0 7 4 7 6 5 5 5
5 0 8 9 0 8 7 0 5 4
8 5 9 7 8 8 9 6 0 8
8 4 8 5 7 6 9 6 0 0
8 7 0 0 9 0 8 8 0 0
6 6 0 0 0 8 8 9 8 9
6 8 0 0 0 0 5 9 4 3
0 0 0 0 0 0 7 4 5 6
9 0 0 0 0 0 0 8 7 6
8 7 0 0 0 0 6 8 4 8


In [98]:
s = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""

day11_part1(s)

1656

In [99]:
with open("./2021/day11_input.txt") as f:
    res = day11_part1(f.read())
res

1661

In [100]:
s = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""

day11_part2(s)

195

In [101]:
with open("./2021/day11_input.txt") as f:
    res = day11_part2(f.read())
res

334

## Day 12: Finding paths

In [9]:
import collections
import itertools

AdjList = dict[str, list[str]]

def day12_parse(s: str) -> AdjList:
    d = collections.defaultdict(list)
    pairs = [line.strip().split("-") for line in s.split("\n") if line.strip()]
    for a, b in pairs:
        d[a].append(b)
        d[b].append(a)
    return dict(d)


def day12_part1(s: str) -> int:
    adj = day12_parse(s)

    def is_smallcave(ss: str):
        return ss != 'start' and ss != 'end' and ss.islower()

    traj = ('start', )
    stack = [traj]
    results = set()

    while stack:
        traj = stack.pop()
        curr = traj[-1]
        if curr == 'end':
            results.add(traj)
            continue

        for x in adj[curr]:
            if x == 'start' or (is_smallcave(x) and (x in traj)):
                continue
            next_traj = tuple(itertools.chain(traj, [x]))
            stack.append(next_traj)

    # print(f"{results = }")
    return len(results)



In [10]:
s = """
start-A
start-b
A-c
A-b
b-d
A-end
b-end
"""
day12_part1(s)

10

In [11]:
s = """
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
"""
day12_part1(s)

19

In [13]:
s = """
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
"""
day12_part1(s)

226

In [14]:
with open("./2021/day12_input.txt") as f:
    res = day12_part1(f.read())
res

3887

In [19]:
def day12_part2(s: str) -> int:
    adj = day12_parse(s)

    def is_smallcave(ss: str):
        return ss != 'start' and ss != 'end' and ss.islower()

    def is_good(x, traj) -> bool:
        if x == "start":
            return False
        
        if x == 'end':
            return True
        
        if x.isupper():
            return True
        
        visited_smallcaves = [c for c in traj if is_smallcave(c)]
        # if you've already visited a small cave twice
        if len(visited_smallcaves) > len(set(visited_smallcaves)):
            return x not in traj
        
        return True


    traj = ('start', )
    stack = [traj]
    results = set()

    while stack:
        traj = stack.pop()
        curr = traj[-1]
        if curr == 'end':
            results.add(traj)
            continue

        for x in adj[curr]:
            if is_good(x, traj):
                next_traj = tuple(itertools.chain(traj, [x]))
                stack.append(next_traj)

    # print(f"{results = }")
    return len(results)


In [20]:
s = """
start-A
start-b
A-c
A-b
b-d
A-end
b-end
"""
day12_part2(s)

36

In [21]:
with open("./2021/day12_input.txt") as f:
    res = day12_part2(f.read())
res

104834